<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/JetMoE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/myshell-ai/JetMoE.git

Cloning into 'JetMoE'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 60 (delta 24), reused 38 (delta 12), pack-reused 0
Receiving objects: 100% (60/60), 2.20 MiB | 30.49 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [2]:
!cd JetMoE && pip install -e .

Obtaining file:///content/JetMoE
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/shawntan/scattermoe (to revision main) to /tmp/pip-install-_b3l6qhy/scattermoe_08b60cfd4a55447099b927072ea675c7
  Running command git clone --filter=blob:none --quiet https://github.com/shawntan/scattermoe /tmp/pip-install-_b3l6qhy/scattermoe_08b60cfd4a55447099b927072ea675c7
  Resolved https://github.com/shawntan/scattermoe to commit 5573cab264aaaa69dc60fcd013a7a6f73099c90f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 11.4 MB/s

In [1]:
!pip list | grep jetmoe

jetmoe                           0.0.0                 /content/JetMoE


In [2]:
%cd JetMoE

/content/JetMoE


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, AutoModelForSequenceClassification
from jetmoe import JetMoEForCausalLM, JetMoEConfig, JetMoEForSequenceClassification

AutoConfig.register("jetmoe", JetMoEConfig)
print(JetMoEConfig)

AutoModelForCausalLM.register(JetMoEConfig, JetMoEForCausalLM)
AutoModelForSequenceClassification.register(JetMoEConfig, JetMoEForSequenceClassification)

tokenizer = AutoTokenizer.from_pretrained('jetmoe/jetmoe-8b')
print(tokenizer)

model = AutoModelForCausalLM.from_pretrained('jetmoe/jetmoe-8b')
print(model)

<class 'jetmoe.configuration_jetmoe.JetMoEConfig'>
LlamaTokenizerFast(name_or_path='jetmoe/jetmoe-8b', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

JetMoEForCausalLM(
  (model): JetMoEModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-23): 24 x JetMoEBlock(
        (input_layernorm): JetMoERMSNorm()
        (self_attention): JetMoESdpaAttention(
          (experts): MoE(
            k=2, e=8
            (input_linear): ParallelExperts(num_experts=8, input_size=2048, output_size=2048)
            (output_linear): ParallelExperts(num_experts=8, input_size=2048, output_size=2048)
            (router): top_k_gating(
              k=2, num_experts=8
              (layer): Linear(in_features=2048, out_features=8, bias=False)
            )
          )
          (kv_proj): Linear(in_features=2048, out_features=4096, bias=False)
          (rotary_emb): JetMoERotaryEmbedding()
        )
        (post_attention_layernorm): JetMoERMSNorm()
        (mlp): MoE(
          k=2, e=8
          (input_linear): ParallelExperts(num_experts=8, input_size=2048, output_size=11264)
          (output_linear): ParallelExper

In [4]:
inputs = tokenizer("hello", return_tensors="pt").input_ids
print(inputs)


tensor([[    1,  6312, 28709]])


In [7]:
output_tokens = model.generate(
    inputs,
    max_new_tokens=32,
    do_sample=True,
    temperature=0.2,
    top_p=0.95,
    top_k=50,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)
print(output_tokens)

tensor([[    1,  6312, 28709, 28725,    13, 28710,   506,   264,  2700,   395,
           586, 28705, 28750, 28734, 28734, 28782, 28705, 28770, 28750, 28782,
         28710, 28723,    13, 28710,   506,   264,  2700,   395,   272,  4456,
          2061, 28723,    13, 28710,   506]])


In [9]:
target_predicted = tokenizer.decode(output_tokens[0], skip_special_tokens=False)
print(target_predicted)


<s> hello,
i have a problem with my 2005 325i.
i have a problem with the engine light.
i have


## fine-tuning

搞些数据，微调下吧~